In [ ]:
# Workflow: Outage Scenario Generation

This workflow notebook generates random outage scenarios by creating device status records with telemetry issues. It simulates real-world device failures that would trigger the self-healing networks system.

## Overview
- **Purpose**: Generate realistic outage scenarios for testing
- **Process**: Random device selection and status assignment
- **Output**: Device status records with telemetry issues
- **Integration**: Feeds into the triage and routing system


In [ ]:
## Step 1: Setup and Load Device Data

Load configuration parameters and device inventory data. This provides the foundation for generating realistic outage scenarios.


In [0]:
import random
from yaml import safe_load
from pyspark.sql import functions as F

with open('../params.yml', 'r') as f:
  params = safe_load(f)

CATALOG = params['data_params']['catalog']
SCHEMA = params['data_params']['schema']


In [ ]:
## Step 2: Generate Random Outage Scenarios

Generate random outage scenarios by:
- Randomly selecting 25-75 devices to have telemetry issues
- Assigning "Telemetry Issue" status to selected devices
- Marking remaining devices as "System Online and Functioning"
- Adding timestamps for all status changes


In [0]:
from pyspark.sql.functions import col, row_number, lit, current_timestamp, monotonically_increasing_id, rand, when
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
import random

devices = spark.table(f"{CATALOG}.{SCHEMA}.tower_devices").select(['tower_id', 'device_id', 'device']).orderBy(rand())



# Add a row number column to uniquely identify rows
window_spec = Window.orderBy(monotonically_increasing_id())
devices_with_row_num = devices.withColumn("row_num", row_number().over(window_spec))

# Randomly select between 3-25 rows to have 'Telemetry Issue'
num_issues = random.randint(25, 75)

print(num_issues)
devices_with_status = devices_with_row_num.withColumn(
    "status",
    when(F.col("row_num").between(1, num_issues), "Telemetry Issue").otherwise("System Online and Functioning")
).withColumn('event_timestamp', current_timestamp()).drop("row_num")

devices_with_status.display()
devices_with_status.write.mode('append').insertInto(f"{CATALOG}.{SCHEMA}.device_status")


In [0]:
print(f"{CATALOG}.{SCHEMA}.device_status")